In [1]:
import pandas as pd
import numpy as np

In [3]:
!wget https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv

--2024-04-24 07:48:32--  https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35005739 (33M) [text/plain]
Saving to: ‘unhealthy_full.csv’

unhealthy_full.csv  100%[===================>]  33.38M   162MB/s    in 0.2s    

2024-04-24 07:48:33 (162 MB/s) - ‘unhealthy_full.csv’ saved [35005739/35005739]



In [4]:
data = pd.read_csv("unhealthy_full.csv")

In [5]:
ann_columns = data.columns.tolist()[4:]

In [6]:
', '.join(ann_columns)

'antagonize, condescending, dismissive, generalisation, generalisation_unfair, healthy, hostile, sarcastic'

In [7]:
for col in ann_columns:
    data[col] = data[col].astype(int)

In [13]:
np.random.seed(22)

selected_texts = data['comment'].drop_duplicates().sample(n=1000).tolist()
selected_data = data.loc[data['comment'].isin(selected_texts)]
not_selected_data = data.loc[~data['comment'].isin(selected_texts)]


In [20]:
print(len(selected_texts))
print(data.shape)
print(selected_data.shape)

1000
(227975, 12)
(4925, 12)


In [14]:

bert_labels_full = pd.read_csv('bert_labels_full.csv')

bert_labels_selected = bert_labels_full[bert_labels_full['_unit_id'].isin(selected_data['_unit_id'])]

bert_labels_selected.to_csv('bert_labels_selected.csv', index=False)

In [ ]:
#VALERIA

major_votes_not_selected_data = not_selected_data.groupby('comment')[ann_columns].mean()

major_votes_not_selected_data[major_votes_not_selected_data < 0.5] = 0
major_votes_not_selected_data[major_votes_not_selected_data >= 0.5] = 1
major_votes_not_selected_data = major_votes_not_selected_data.astype(int).reset_index()

random_examples_per_label = {}

for label in ann_columns:
    # Filter 'not_selected_data' for rows where the label column contains label
    label_data = major_votes_not_selected_data[major_votes_not_selected_data[label]==1]

    # Exclude examples already selected for other labels
    if random_examples_per_label:
        already_selected = pd.concat(random_examples_per_label.values())
        label_data = label_data.loc[~label_data.index.isin(already_selected.index)]

    # Select 10 random examples
    random_examples = label_data.sample(n=10, random_state=22)
    random_examples_per_label[label] = random_examples

print(random_examples_per_label)

# 'random_examples_per_label' now contains 10 random examples for each label in 'ann_columns'

In [ ]:
major_votes_data = selected_data.groupby('comment')[ann_columns].mean()

major_votes_data[major_votes_data < 0.5] = 0
major_votes_data[major_votes_data >= 0.5] = 1
major_votes_data = major_votes_data.astype(int).reset_index()

def get_annotations(row):
  labels = [column for column in ann_columns if row[column] == 1]

  # Join the labels into a single string, separated by commas
  return ', '.join(labels)

def get_prompt(row, shots):
    text = row['comment']
    pattern = 'Knowing that a certain person described the following texts with the given attributes (among: "antagonize", "condescending", "dismissive", "generalisation", "unfair generalisation", "healthy", "hostile", "sarcastic")\n\n '
    for label in ann_columns:
      for ex in range(shots):
        pattern = pattern + "text:'"+random_examples_per_label[label].iloc[ex]['comment'] + "' annotation:"+get_annotations(random_examples_per_label[label].iloc[ex])+'\n\n'

    pattern = pattern + "\nwrite which of the attributes: 'antagonize', 'condescending', 'dismissive', 'generalisation', 'unfair generalisation', 'healthy', 'hostile', 'sarcastic' describes a given text? Write your answer in the form of a Python list containing the appropriate attributes. Do not explain yourself.\n\n Text: "+text

    return pattern

for shots in range(2,11,2):
  column_name = f'prompt{shots}'
  major_votes_data[column_name] = major_votes_data.apply(get_prompt, axis=1, args=(shots,))
  print (major_votes_data[column_name])

In [ ]:

def get_labels(row):
    return np.array(ann_columns)[row[ann_columns].values == 1]

major_votes_data['annotation'] = major_votes_data.apply(get_labels , axis=1)

In [ ]:
# List of columns to include in the CSV file
columns_to_include = ['comment', 'annotation']

# Add the dynamically generated prompt columns
for shots in range(2, 11, 2):
    columns_to_include.append(f'prompt{shots}')

# Export the selected columns to CSV
major_votes_data.loc[:, columns_to_include].to_csv("18_uc_prompts_few_shots.csv")